# Generating the DRH Datasets

This code notebook walks through the steps taken to curate the Drug Repurposing Hub (DRH) datasets to train the model. The following steps were performed for each of the top 3 cell lines that were used (A735, MCF7, and PC3):
> 1. Obtain the names of all drugs have indicated listed in DRH
> 2. Obtain all LINCS signatures for those drugs
> 3. Add the TAS scores to the drugs
> 4. Calculate the spearman correlation between all pairs of drugs
> 5. Create a dataframe that contains all drug1 to drug2 indications possible, the TAS of the two drugs, and the spearman correlation between the drugs

## *1. Select all gene signatures of compounds in Drug Repurposing Hub*

This was done by selecting the gene signature that were:
1. Treated with a small compound
2. Treated by one of the top three cell lines used (A375, MCF7, or PC3)
3. Exhibited a high transcriptional response (TAS > 0.2)
4. Indicated for a treatment based on Drug Repurposing Hub

To remove duplicate gene signatures treated with the same compound, only the gene signature with the highest TAS score was kept.

In [1]:
# import data manipulation tool
import pandas as pd

In [2]:
# read in the LINCS signatures
file_path = "./ref_data/GSE70138_Broad_LINCS_sig_info.txt"
LINCS_sigs = pd.read_csv(file_path, index_col=0, sep="\t")
# keep only the signatures treated with small compounds
LINCS_sigs = LINCS_sigs[LINCS_sigs["pert_type"] == "trt_cp"]
# keep only the signatures treated by A375, MCF7, or PC3
cell_lines = ["A375", "MCF7", "PC3"]
LINCS_sigs = LINCS_sigs[LINCS_sigs["cell_id"].isin(cell_lines)]
LINCS_sigs.head()

,pert_id,pert_iname,pert_type,cell_id,pert_idose,pert_itime,distil_id
sig_id,,,,,,,
LJP005_A375_24H:A07,BRD-K76908866,CP-724714,trt_cp,A375,10.0 um,24 h,LJP005_A375_24H_X1_B19:A07|LJP005_A375_24H_X2_...
LJP005_A375_24H:A08,BRD-K76908866,CP-724714,trt_cp,A375,3.33 um,24 h,LJP005_A375_24H_X1_B19:A08|LJP005_A375_24H_X2_...
LJP005_A375_24H:A09,BRD-K76908866,CP-724714,trt_cp,A375,1.11 um,24 h,LJP005_A375_24H_X1_B19:A09|LJP005_A375_24H_X2_...
LJP005_A375_24H:A10,BRD-K76908866,CP-724714,trt_cp,A375,0.37 um,24 h,LJP005_A375_24H_X1_B19:A10|LJP005_A375_24H_X2_...
LJP005_A375_24H:A11,BRD-K76908866,CP-724714,trt_cp,A375,0.12 um,24 h,LJP005_A375_24H_X1_B19:A11|LJP005_A375_24H_X2_...


In [3]:
# read in the metrics file containing the tas scores of gene signatures
file_path = "./ref_data/GSE70138_Broad_LINCS_sig_metrics_2017-03-06.txt"
sig_metrics = pd.read_csv(file_path, index_col="sig_id", sep="\t")
# store tas scores of the sig ids treated with compounds
tas = sig_metrics.loc[LINCS_sigs.index, "tas"]
# add the tas scores to signature information
LINCS_sigs = pd.concat([LINCS_sigs, tas], axis=1)
LINCS_sigs.head()

,pert_id,pert_iname,pert_type,cell_id,pert_idose,pert_itime,distil_id,tas
sig_id,,,,,,,,
LJP005_A375_24H:A07,BRD-K76908866,CP-724714,trt_cp,A375,10.0 um,24 h,LJP005_A375_24H_X1_B19:A07|LJP005_A375_24H_X2_...,0.442109
LJP005_A375_24H:A08,BRD-K76908866,CP-724714,trt_cp,A375,3.33 um,24 h,LJP005_A375_24H_X1_B19:A08|LJP005_A375_24H_X2_...,0.458614
LJP005_A375_24H:A09,BRD-K76908866,CP-724714,trt_cp,A375,1.11 um,24 h,LJP005_A375_24H_X1_B19:A09|LJP005_A375_24H_X2_...,0.114044
LJP005_A375_24H:A10,BRD-K76908866,CP-724714,trt_cp,A375,0.37 um,24 h,LJP005_A375_24H_X1_B19:A10|LJP005_A375_24H_X2_...,0.229162
LJP005_A375_24H:A11,BRD-K76908866,CP-724714,trt_cp,A375,0.12 um,24 h,LJP005_A375_24H_X1_B19:A11|LJP005_A375_24H_X2_...,0.188851


In [4]:
# sort values the signatures from highest to lowest tas scores
LINCS_sigs = LINCS_sigs.sort_values(by="tas", ascending=False)
# remove duplicate signatures treated with same compound and cell line by keeping the one with the highest tas
LINCS_sigs = LINCS_sigs.drop_duplicates(subset=["pert_iname", "cell_id"], keep="first")
# remove unnessary columns
cols2keep = ["pert_id", "pert_iname", "cell_id", "tas"]
LINCS_sigs = LINCS_sigs[cols2keep]
LINCS_sigs.head()

,pert_id,pert_iname,cell_id,tas
sig_id,,,,
REP.A002_A375_24H:G18,BRD-K50691590,bortezomib,A375,0.870794
REP.A002_A375_24H:K11,BRD-K60230970,MG-132,A375,0.868866
REP.A024_A375_24H:N14,BRD-K78659596,ixazomib,A375,0.853718
REP.A013_A375_24H:D08,BRD-K37890730,camptothecin,A375,0.853287
REP.A002_A375_24H:L19,BRD-K17743125,belinostat,A375,0.850827


In [5]:
# read in the DRH indications
file_path = "./ref_data/drug_ind/normalized/collapsed_drug_repo_lincs_ind.txt"
LINCS_drug_repo = pd.read_csv(file_path)
# set index as the name of the compound
LINCS_drug_repo.set_index("pert_iname", inplace=True)
LINCS_drug_repo.head()

,indication
pert_iname,
5-aminolevulinic-acid,"Keratosis, Actinic|Glioma"
L-citrulline,Hypertension|Erectile Dysfunction
SN-38,Colorectal Neoplasms
abacavir,HIV
abiraterone-acetate,Prostatic Neoplasms


In [6]:
# keep only gene signatures treated with compounds that have an indication
shared_cps = LINCS_drug_repo.index
LINCS_drh_sigs = LINCS_sigs[LINCS_sigs["pert_iname"].isin(shared_cps)]
LINCS_drh_sigs.head()

,pert_id,pert_iname,cell_id,tas
sig_id,,,,
REP.A002_A375_24H:G18,BRD-K50691590,bortezomib,A375,0.870794
REP.A024_A375_24H:N14,BRD-K78659596,ixazomib,A375,0.853718
REP.A002_A375_24H:L19,BRD-K17743125,belinostat,A375,0.850827
REP.A025_A375_24H:L10,BRD-K76674262,homoharringtonine,A375,0.837631
REP.A025_A375_24H:C13,BRD-K92213669,lomitapide,A375,0.830835


In [7]:
# total number of compounds available for each cell line
LINCS_drh_sigs.groupby("cell_id")["pert_iname"].size()

cell_id
A375    800
MCF7    799
PC3     799
Name: pert_iname, dtype: int64

In [8]:
# initialize a dictionary to store all the gene signatures separated by cell line
DRH_sigs = {}

# for each each cell
for cell_line in cell_lines:
    # store the gene signatures treated by that cell line in dictionary
    DRH_sigs[cell_line] = LINCS_drh_sigs[LINCS_drh_sigs["cell_id"] == cell_line]

DRH_sigs["MCF7"].head()

,pert_id,pert_iname,cell_id,tas
sig_id,,,,
REP.A020_MCF7_24H:G13,BRD-K50691590,bortezomib,MCF7,0.826145
REP.A010_MCF7_24H:L03,BRD-K14619660,altrenogest,MCF7,0.812819
REP.A015_MCF7_24H:E08,BRD-K56844688,epirubicin,MCF7,0.807126
REP.A020_MCF7_24H:L01,BRD-A45889380,mepacrine,MCF7,0.806568
REP.A010_MCF7_24H:I02,BRD-K13646352,midostaurin,MCF7,0.799386


## *2. Calculate the spearman correlation across landmark genes between all compounds of interest*

DRH datasets will include:
1. A pair of drugs
2. The TAS of each drug
3. The spearman correlation between their gene signatures
4. Whether the two drugs share an indication (based on data available from the Drug Repurposing Hub)

In [9]:
# read in the gene information
file_path = "./ref_data/GSE70138_Broad_LINCS_gene_info_2017-03-06.txt"
lm_genes = pd.read_csv(file_path, sep="\t", dtype=str)
# select only the gene ids of the landmark genes
lm_gene_ids = lm_genes.loc[lm_genes["pr_is_lm"] == "1", "pr_gene_id"]
# number of landmark genes
len(lm_gene_ids)

978

In [10]:
# import data manipulation packages
import cmapPy.pandasGEXpress.GCToo as GCToo
from cmapPy.pandasGEXpress.parse import parse
import cmapPy.pandasGEXpress.subset_gctoo as sg

In [11]:
### function to generate a dataframe containing spearman correlations between any pair of drugs
def gen_spearman_corrs(sigs_by_cell_line, cell_line):
    ## store the gene signatures for that cell line
    sig_info = sigs_by_cell_line[cell_line]
    ## obtain the z-scores for all landmark genes for all gene signatures
    gene_sigs = parse(LINCS_data, rid=lm_gene_ids, cid=sig_info.index)
    ## replace the signature ids with the names of the compounds
    all_cps = sig_info.loc[gene_sigs.data_df.columns, "pert_iname"]
    gene_sigs.data_df.columns = all_cps
    ## perform pairwise spearman correlation
    pairwise_corrs = gene_sigs.data_df.corr(method="spearman")
    
    ## set the index tas scores dataframe as the name of the compound
    tas_scores = sig_info.reset_index().set_index("pert_iname")
    
    ## initialize a new dataframe to store all the spearman correlations
    spearman_corrs = pd.DataFrame()
    ## iterate through all compounds
    for cp in all_cps:
        # get all the spearman correlations to the drug of interest
        cp_corrs = pairwise_corrs[cp].drop(cp).reset_index()
        # rename columns
        cp_corrs.columns = ["drug2", "spearman corr"]
        # add a column with the compound of interest
        cp_corrs.insert(0, "drug1", cp)
        # add a column with the tas of all the drug 1s
        drug1_tas = tas_scores.loc[cp, "tas"]
        cp_corrs.insert(1, "drug1 tas", drug1_tas)
        # add a column with the tas of all the drug 2s
        drug2_tas = tas_scores.loc[cp_corrs["drug2"], "tas"].tolist()
        cp_corrs.insert(3, "drug2 tas", drug2_tas)
        # add this to dataframe for all compounds
        spearman_corrs = pd.concat([spearman_corrs, cp_corrs])
        
    return spearman_corrs

In [12]:
# location of file containing the gene signatures
LINCS_data = "./ref_data/LINCS_dataset.gctx"
# get the spearman correlations between all pairs of drugs for one cell line
DRH_spearman_corrs = gen_spearman_corrs(DRH_sigs, "MCF7")
DRH_spearman_corrs.head()

,drug1,drug1 tas,drug2,drug2 tas,spearman corr
0,aminoglutethimide,0.205099,oxaprozin,0.301089,0.283938
1,aminoglutethimide,0.205099,physostigmine,0.224861,-0.066357
2,aminoglutethimide,0.205099,blonanserin,0.132461,0.227929
3,aminoglutethimide,0.205099,josamycin,0.319157,-0.087767
4,aminoglutethimide,0.205099,propafenone,0.309858,0.384215


## *3. Generate the relevant data using data from DRH*

In [13]:
### function to add whether the indication is a known indication of drug1
def add_known_ind(row):
    ## get the indications of drug1
    drug1_ind = row["drug1 indications"].split("|")
    ## get the indication of drug2
    ind = row["indication"]
    
    ## return whether that indication is a known indication of drug1
    known_ind = ind in drug1_ind
    return known_ind

In [14]:
### function to add indications to spearman correlations
def gen_dataset(spearman_corrs, drug_ind):
    ## add a column with the indications for the drug 1s
    drug1_inds = drug_ind.loc[spearman_corrs["drug1"], "indication"].tolist()
    spearman_corrs.insert(2, "drug1 indications", drug1_inds)
    
    ## split the "indication" column and create a new series with duplicated rows for each indication
    ind = drug_ind["indication"].str.split("|", expand=True).stack().reset_index(level=1, drop=True).rename("indication")
    ## create a new dataframe that maps each drug to one indication
    drug2_ind = drug_ind.drop(columns="indication").join(ind).reset_index()
    ## keep only the name of drug, and the indication
    drug2_ind = drug2_ind[["pert_iname", "indication"]]
    ## create a new dataframe separated by each indication of drug 2
    train_data = pd.merge(spearman_corrs, drug2_ind, left_on="drug2", right_on="pert_iname", how="left")
    ## add a column containing whether the indication is a known indication of drug1
    train_data["known ind"] = train_data.apply(add_known_ind, axis=1)
    ## remove columns
    train_data.drop(columns=["drug1 indications", "pert_iname"], axis=1, inplace=True)
    
    return train_data

In [15]:
# generate the DRH dataset for one cell line
DRH_data = gen_dataset(DRH_spearman_corrs, LINCS_drug_repo)
DRH_data.head()

,drug1,drug1 tas,drug2,drug2 tas,spearman corr,indication,known ind
0,aminoglutethimide,0.205099,oxaprozin,0.301089,0.283938,"Arthritis, Rheumatoid",False
1,aminoglutethimide,0.205099,oxaprozin,0.301089,0.283938,Osteoarthritis,False
2,aminoglutethimide,0.205099,physostigmine,0.224861,-0.066357,Glaucoma,False
3,aminoglutethimide,0.205099,physostigmine,0.224861,-0.066357,Hypotension,False
4,aminoglutethimide,0.205099,physostigmine,0.224861,-0.066357,Alzheimer Disease,False


## *5. Generate DRH and clinical trials datasets for all cell lines*

In [ ]:
### initialize a dictionary to store the DRH datasets
DRH_data = {}

### for each cell line
for cell_line in cell_lines:
    ## generate the spearman correlations for each pair of drugs based on data from DRH
    DRH_corrs = spearman_corrs = gen_spearman_corrs(DRH_sigs, cell_line)
    ## generate the training datasets
    cell_line_data = gen_dataset(DRH_corrs, LINCS_drug_repo)
    
    ## sort the rows from highest to lowest spearman correlation
    cell_line_data = cell_line_data.sort_values("spearman corr", ascending=False)
    ## remove duplicate drug-indication pairs by picking the one with the highest spearman correlation to drug2
    unique_data = cell_line_data.drop_duplicates(subset=["drug1", "indication"], keep="first")
    ## store the dataframe in the dictionary for all cell lines
    DRH_data[cell_line] = unique_data
    
    ## save the dataframe for future access
    # file_path = "./DRH_clin_data/DRH_data/" + cell_line + "_unique_ind.txt"
    # unique_data.to_csv(file_path, index=False)

DRH_data["MCF7"].head()